In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
def init_weights(shape):
  init_random_dist = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(init_random_dist)

In [0]:
def init_bias(shape):
  init_random_dist = tf.constant(0.1, shape=shape)
  return tf.Variable(init_random_dist)

In [0]:
def conv2d(x,W):
  return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

In [0]:
def max_pool_2by2(x):
  return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [0]:
def convolutional_layer(input_x, shape):
  W = init_weights(shape)
  b = init_bias([ shape[3] ])
  
  return tf.nn.relu(conv2d(input_x, W)+b)

In [0]:
def normal_full_layer(input_layer, size):
  input_size = int(input_layer.get_shape()[1])
  W = init_weights([input_size, size])
  b = init_bias([size])
  return tf.matmul(input_layer, W) + b

In [0]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_true = tf.placeholder(tf.float32, shape=[None, 10])
hold_prob = tf.placeholder(tf.float32)

In [0]:
x_image = tf.reshape(x, [-1,28,28,1])

In [11]:
conv_1 = convolutional_layer(x_image, shape=[5,5,1,32])
conv_2 = convolutional_layer(conv_1, shape=[5,5,32,32])

conv_2_pool = max_pool_2by2(conv_1)

conv_2_dropout = tf.nn.dropout(conv_2_pool, rate=hold_prob)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
conv_3 = convolutional_layer(conv_2_dropout, shape=[5,5,32,64])
conv_4 = convolutional_layer(conv_3, shape=[5,5,64,64])

conv_4_pool = max_pool_2by2(conv_4)

conv_4_dropout = tf.nn.dropout(conv_4_pool, rate=hold_prob)

In [0]:
conv_4_flat = tf.reshape(conv_4_dropout, [-1, 7*7*64])
fc_layer = tf.nn.relu(normal_full_layer(conv_4_flat, 1024))

In [0]:
dropout = tf.nn.dropout(fc_layer, rate=hold_prob)

In [0]:
y_pred = normal_full_layer(dropout, 10)

In [16]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred ))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [0]:
init = tf.global_variables_initializer()

In [19]:
steps = 5000

with tf.Session() as sess:
  
  sess.run(init)
  
  for i in range(steps+1):
    batch_x, batch_y = mnist.train.next_batch(50)
    
    sess.run(train, feed_dict={x:batch_x, y_true:batch_y, hold_prob:0.33})
    
    
    if i % 500 == 0:
      matches = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true, 1))
      acc = tf.reduce_mean(tf.cast(matches, tf.float32))
      result = sess.run(acc, feed_dict={x:mnist.test.images, y_true:mnist.test.labels, hold_prob:0.0})
      
      print('STEP {}: Accuracy: {}'.format(i,result))

STEP 0: Accuracy: 0.12929999828338623
STEP 500: Accuracy: 0.965399980545044
STEP 1000: Accuracy: 0.9757999777793884
STEP 1500: Accuracy: 0.9811000227928162
STEP 2000: Accuracy: 0.986299991607666
STEP 2500: Accuracy: 0.987500011920929
STEP 3000: Accuracy: 0.9868000149726868
STEP 3500: Accuracy: 0.9883999824523926
STEP 4000: Accuracy: 0.9905999898910522
STEP 4500: Accuracy: 0.9907000064849854
STEP 5000: Accuracy: 0.9926000237464905
